# RFI Band Analysis — Interference Impact Across Frequency Bands

This notebook produces four plots that progressively stress-test victim satellite links across S, X, Ku, K, and Ka bands under increasing interference levels.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys

# Ensure local package visibility
current_dir = os.path.dirname(os.path.abspath(""))
sys.path.append(current_dir)
sys.path.append(os.path.join(current_dir, "rfi"))

from rfi.scenario import (
    run_multi_entry_rfi_scenario,
    VICTIM_BANDS,
    INTERFERENCE_SCENARIOS,
)

SAMPLES = 5000
band_names = list(VICTIM_BANDS.keys())
print("Setup complete.")

---
## Plot 1 — Baseline SNR (No Interference)

**Purpose:** Sanity check — verifies the antenna-theory and link-budget implementation.  
This plot is the **anchor** against which all degradation is measured.

In [ ]:
baseline_snr = []
for band in band_names:
    cfg = VICTIM_BANDS[band]
    res = run_multi_entry_rfi_scenario(
        band_params=cfg,
        interferer_list=INTERFERENCE_SCENARIOS["Weak"],
        time_sim_samples=SAMPLES,
    )
    baseline_snr.append(res["Baseline SNR (dB)"])

colors = ["#2196F3", "#4CAF50", "#FF9800", "#9C27B0", "#F44336"]

fig, ax = plt.subplots(figsize=(9, 5))
bars = ax.bar(band_names, baseline_snr, color=colors, edgecolor="black", linewidth=0.6)

for bar, val in zip(bars, baseline_snr):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.5,
            f"{val:.1f}", ha="center", va="bottom", fontsize=10, fontweight="bold")

ax.set_xlabel("Frequency Band", fontsize=12)
ax.set_ylabel("Baseline SNR (dB)", fontsize=12)
ax.set_title("Plot 1 — Baseline SNR (No Interference)", fontsize=14, fontweight="bold")
ax.grid(axis="y", alpha=0.3, linestyle="--")
ax.set_ylim(0, max(baseline_snr) * 1.2)
plt.tight_layout()
plt.show()

---
## Plot 2 — Weak Interference (ΔSNR)

**Purpose:** Shows first-order sensitivity.  
Ideally shows almost **no degradation** — confirms that the system is robust under mild RFI.

In [ ]:
weak_loss = []
for band in band_names:
    cfg = VICTIM_BANDS[band]
    res = run_multi_entry_rfi_scenario(
        band_params=cfg,
        interferer_list=INTERFERENCE_SCENARIOS["Weak"],
        time_sim_samples=SAMPLES,
    )
    weak_loss.append(res["SNR Loss (dB)"])

fig, ax = plt.subplots(figsize=(9, 5))
bars = ax.bar(band_names, weak_loss, color="#81D4FA", edgecolor="black", linewidth=0.6)

for bar, val in zip(bars, weak_loss):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.02,
            f"{val:.2f}", ha="center", va="bottom", fontsize=10, fontweight="bold")

ax.set_xlabel("Frequency Band", fontsize=12)
ax.set_ylabel("ΔSNR (dB)", fontsize=12)
ax.set_title("Plot 2 — SNR Degradation under Weak Interference", fontsize=14, fontweight="bold")
ax.grid(axis="y", alpha=0.3, linestyle="--")
ax.set_ylim(0, max(weak_loss) * 1.5 if max(weak_loss) > 0 else 1)
plt.tight_layout()
plt.show()

---
## Plot 3 — Moderate Interference (ΔSNR)

**Purpose:** The **main comparison** — shows real frequency-dependent behavior under realistic RFI conditions.

In [ ]:
moderate_loss = []
for band in band_names:
    cfg = VICTIM_BANDS[band]
    res = run_multi_entry_rfi_scenario(
        band_params=cfg,
        interferer_list=INTERFERENCE_SCENARIOS["Moderate"],
        time_sim_samples=SAMPLES,
    )
    moderate_loss.append(res["SNR Loss (dB)"])

fig, ax = plt.subplots(figsize=(9, 5))
bars = ax.bar(band_names, moderate_loss, color="#FFB74D", edgecolor="black", linewidth=0.6)

for bar, val in zip(bars, moderate_loss):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.05,
            f"{val:.2f}", ha="center", va="bottom", fontsize=10, fontweight="bold")

ax.set_xlabel("Frequency Band", fontsize=12)
ax.set_ylabel("ΔSNR (dB)", fontsize=12)
ax.set_title("Plot 3 — SNR Degradation under Moderate Interference", fontsize=14, fontweight="bold")
ax.grid(axis="y", alpha=0.3, linestyle="--")
ax.set_ylim(0, max(moderate_loss) * 1.3)
plt.tight_layout()
plt.show()

---
## Plot 4 — Strong Interference (ΔSNR)

**Purpose:** Stress test — reveals saturation, nonlinear trends, or reversals.  
This plot answers: **"What breaks first?"**

In [ ]:
strong_loss = []
for band in band_names:
    cfg = VICTIM_BANDS[band]
    res = run_multi_entry_rfi_scenario(
        band_params=cfg,
        interferer_list=INTERFERENCE_SCENARIOS["Strong"],
        time_sim_samples=SAMPLES,
    )
    strong_loss.append(res["SNR Loss (dB)"])

fig, ax = plt.subplots(figsize=(9, 5))
bars = ax.bar(band_names, strong_loss, color="#E57373", edgecolor="black", linewidth=0.6)

for bar, val in zip(bars, strong_loss):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.1,
            f"{val:.2f}", ha="center", va="bottom", fontsize=10, fontweight="bold")

ax.set_xlabel("Frequency Band", fontsize=12)
ax.set_ylabel("ΔSNR (dB)", fontsize=12)
ax.set_title("Plot 4 — SNR Degradation under Strong Interference (Stress Test)", fontsize=14, fontweight="bold")
ax.grid(axis="y", alpha=0.3, linestyle="--")
ax.set_ylim(0, max(strong_loss) * 1.3)
plt.tight_layout()
plt.show()